In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
# import matplotlib.image as mpimg

In [2]:
# import imgaug as ia
# from imgaug import augmenters as iaa

In [3]:
import pandas as pd
import os
import sys
from datetime import datetime
import time
from pprint import pprint

In [4]:
# attention
am_I_using_colab = False

In [5]:
# if am_I_using_colab:
#     !pip install -U tensorflow-addons
    
# import tensorflow_addons as tfa
# import tqdm

In [6]:
# plt.ioff()  # interactive mode off
# plt.ion()  # interactive mode on

# %pylab inline

np.random.seed(1728)
# ia.random.seed(1728)

In [7]:
if am_I_using_colab:
    ###
    from google.colab import drive
    drive.mount('/content/drive')

    print(os.getcwd())
    print(os.listdir())

    ###
    temp_dir = '/content/temp'
    try:
        os.mkdir(temp_dir)
    except:
        pass
    print(os.listdir(temp_dir))

    ###
    dataset_filepath = f'{temp_dir:s}/dataset_v1.7z'
    import gdown
    gdown.download(
        r'https://drive.google.com/uc?id=15kAUrJvaJy54xoXIwPZqtkBQGccjOR5k',
        output=dataset_filepath,
        quiet=False,
        )

    ###
    !pip install py7zr
    import py7zr
    with py7zr.SevenZipFile(dataset_filepath, 'r') as archive:
        archive.extractall(path="./dataset_v1")

    ###


In [8]:
import time

def mulai_hitung_waktu():
    global waktu_mulai
    waktu_mulai = time.time()

def cetak_lama_waktu():
    global waktu_mulai
    hasil_detik = abs(waktu_mulai - time.time())
    hasil_milidetik = hasil_detik * 1000
    
    menit = hasil_detik / 60
    detik = hasil_detik % 60
    
    menit_detik = str(int(menit)) + 'm' + ':' + str(int(detik)) + 's'
    
    print('-----\n(%.2f ms) == (%s)\n-----' % (hasil_milidetik, menit_detik))
    del waktu_mulai


# cara pakai
# ----------
mulai_hitung_waktu()  ###

time.sleep(100/1000)  # time-consuming computing here

cetak_lama_waktu()  ###

-----
(107.01 ms) == (0m:0s)
-----


In [9]:
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print("Num GPUs Available: %d Physical GPUs, %d Logical GPU" % (len(gpus), len(logical_gpus)))
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

Num GPUs Available: 1 Physical GPUs, 1 Logical GPU


In [10]:
if am_I_using_colab:
    WORK_DIR = '/content'
    TRAIN_SET_PATH = f'{WORK_DIR}/dataset_v1'
    # TEST_SET_PATH = f''

    DRIVE_DIR = '/content/drive/MyDrive/MyNotebook/MySkripsi'
    CHECKPOINTS_DIR = f'{WORK_DIR}/checkpoints'
    FINAL_EPOCH_DIR = f'{WORK_DIR}/weights and models'

else:
    WORK_DIR = r'D:\MaskTheFace\datasets\_temp'
    TRAIN_SET_PATH = r'D:\MaskTheFace\datasets\_temp\_v1'
    TEST_SET_PATH = r'D:\MaskTheFace\datasets\_temp\_v1 - test only'

    CHECKPOINTS_DIR = r'D:\MaskTheFace\datasets\_temp\checkpoints'
    FINAL_EPOCH_DIR = r'D:\MaskTheFace\datasets\_temp\weights and models'

directories = [
    CHECKPOINTS_DIR,
    FINAL_EPOCH_DIR
]

for dir in directories:
    print(dir)
    try:
        os.mkdir(dir)
        print('success')
    except FileExistsError:
        print('exist')
    except:
        print('error')

# Set working directory
os.chdir(WORK_DIR)

print('current work dir:')
print(os.getcwd())
print()
print('listdir WORK_DIR:')
pprint(os.listdir('./'))

D:\MaskTheFace\datasets\_temp\checkpoints
exist
D:\MaskTheFace\datasets\_temp\weights and models
exist
current work dir:
D:\MaskTheFace\datasets\_temp

listdir WORK_DIR:
['Augmented Images - Masked',
 'Augmented Images - Masked.zip',
 'Augmented Images - Unmasked',
 'Augmented Images - Unmasked.zip',
 'checkpoints',
 'main, train set - crop masked resized',
 'main, train set - crop masked resized.zip',
 'main, train set - crop resized',
 'main, train set - crop resized.zip',
 'Model tracking and summary.xlsx',
 'note.txt',
 'thumbnails - augmented masked.zip',
 'thumbnails - augmented unmasked.zip',
 'verbose.mobilenetv2_1.00_128-16-fc12.txt',
 'verbose.mobilenetv2_1.00_128-imagenet128-16-12.txt',
 'verbose.mobilenetv2_1.00_128-imagenet128-16-fc12.txt',
 'verbose.mobilenetv2_1.00_128-imagenet128-32-12.txt',
 'verbose.mobilenetv2_1.00_32-16-12.txt',
 'verbose.mobilenetv2_1.00_32-64-12.txt',
 'verbose.mobilenetv2_1.00_32-64-fc12.txt',
 'verbose.mobilenetv2_1.00_32-imagenet224-16-12.txt',

In [11]:
names = os.listdir(TRAIN_SET_PATH)

length_dict = {}
for name in names:
    samples = f'{TRAIN_SET_PATH:s}/{name:s}'
    length_dict[name] = len(os.listdir(samples))

n_min = min(length_dict.values())
n_max = max(length_dict.values())
n_sum = sum(length_dict.values())

print(f'{len(names):d} names')
display(length_dict)
print(f"max: {n_min:d}")
print(f"min: {n_max:d}")
print(f"total: {n_sum:d}")

# del n_min, n_max, n_sum, length_dict

12 names


{'Andhika': 1260,
 'Ardiyan': 1260,
 'Artik': 1260,
 'Ballya': 1260,
 'Bina': 1260,
 'Buyung': 1260,
 'Kresna': 1260,
 'Mhartian': 1260,
 'RaihanA': 1260,
 'Syifa': 1260,
 'Taufik': 1260,
 'Yandi': 1260}

max: 1260
min: 1260
total: 15120


In [12]:
validation_split = 0.4
total_train = round(n_sum * (1 - validation_split))
total_val = n_sum - total_train

print(f'train images: {total_train:d}')
print(f'validation images: {total_val:d}')

train images: 9072
validation images: 6048


In [13]:
def new_model(input_size, batch_size, weights=None, dense=False):
    base_model = keras.applications.MobileNetV2(
        weights=weights,  # Load weights pre-trained on ImageNet.
        input_shape=(input_size, input_size, 3),  # recommended  because trained in (224, 224, 3) ImageNet
        alpha=1.0,  # network width multiplier, default 1.0 in the MobileNetV2 paper
        include_top=False)  # Do not include the ImageNet classifier at the top.

    if weights == 'imagenet':
        base_model.trainable = False  # freeze the base model.
    else:
        base_model.trainable = True

    # Create a new model on top.
    inputs = keras.Input(shape=(input_size, input_size, 3))

    # We make sure that the base_model is running in inference mode here,
    # by passing `training=False`.
    # This is important for fine-tuning, as you will learn in a few paragraphs.

    if weights == 'imagenet':
        x = base_model(inputs, training=False)  # freeze the base model.
    else:
        x = base_model(inputs, training=True)

    # Convert features of shape `base_model.output_shape[1:]` to vectors
    x = keras.layers.GlobalAveragePooling2D()(x)

    if bool(dense) == True:
        # Regularize with dropout
        x = keras.layers.Dropout(0.1)(x)
        x = keras.layers.Dense(256,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
        x = keras.layers.Dropout(0.1)(x)
        x = keras.layers.Dense(256,activation='relu')(x) #dense layer 2
        x = keras.layers.Dropout(0.1)(x)
        x = keras.layers.Dense(128,activation='relu')(x) #dense layer 3
        x = keras.layers.Dropout(0.05)(x)

    else:
        # Regularize with dropout
        x = keras.layers.Dropout(0.2)(x)

    # A Dense classifier with a single unit (binary classification)
    outputs = keras.layers.Dense(12, activation="softmax")(x)

    if weights == 'imagenet':
        if input_size in [96, 128, 192, 224]:
            name = f'mobilenetv2_1.00_{input_size:d}-imagenet{input_size:d}-{batch_size:d}'
        else:
            name = f'mobilenetv2_1.00_{input_size:d}-imagenet224-{batch_size:d}'
    else:
        name = f'mobilenetv2_1.00_{input_size:d}-{batch_size:d}'

    if bool(dense) == True:
        name = f'{name:s}-fc12'
    else:
        name = f'{name:s}-12'

    model = keras.Model(inputs=inputs, outputs=outputs, name=name)
    print(model.name)
    return model

In [14]:
def consecutiveModelTraining(
    input_size,  # [32, 64, 96, 128] or divisible by 32
    batch_size,  # [16, 32, 64] or power of 
    weights,     # [None, 'imagenet']
    dense):      # [True, False]
    
    
    ### create train/val generator
    image_generator = keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        rotation_range=5,
        width_shift_range=1.05,
        height_shift_range=1.05,
        brightness_range=(0.75, 1.25),
        shear_range = 0.075,
        zoom_range=0.2,
        fill_mode = 'nearest',
        horizontal_flip=True,
        # vertical_flip=True,
        validation_split=validation_split,
        # preprocessing_function=seq.augment_image
    )
    train_data_gen = image_generator.flow_from_directory(
        seed=1728,
        subset='training',
        batch_size=batch_size,
        directory=TRAIN_SET_PATH,
        shuffle=True,
        target_size=(input_size, input_size),
        interpolation='bicubic',
        class_mode='categorical')

    val_data_gen = image_generator.flow_from_directory(
        seed=1728,
        subset='validation',
        batch_size=batch_size,
        directory=TRAIN_SET_PATH,
        shuffle=True,
        target_size=(input_size, input_size),
        interpolation='bicubic',
        class_mode='categorical')
    
    
    ### create model
    model = new_model(input_size=input_size, batch_size=batch_size, weights=weights, dense=dense)
    print()
    print()
    
    
    ### model optimizer, loss fn, metrics
    # Optimizer's scheduler: ExponentialDecay
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-2,
        decay_steps=10000,
        decay_rate=0.9)
    
    optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)
    loss_fn = 'categorical_crossentropy'
    metrics=[
        'categorical_accuracy',
        keras.metrics.TopKCategoricalAccuracy(k=3, name='top-3'),
        keras.metrics.AUC(name='ROC-AUC', curve='ROC'),
        keras.metrics.AUC(name='PR-AUC', curve='PR'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.TruePositives(name='TP'),
        keras.metrics.TrueNegatives(name='TN'),
        keras.metrics.FalsePositives(name='FP'),
        keras.metrics.FalseNegatives(name='FN'),]

    model.compile(
        optimizer=optimizer,
        loss=loss_fn,
        metrics=metrics)
    
    model.summary()
    print()
    print()
    
    
    ###
    params = {
        'total_train' : total_train,
        'total_val' : total_val,
        'batch_size' : batch_size,
        'steps_per_epoch' : total_train//batch_size,
        'validation_steps' : total_val//batch_size,
        'epochs' : epochs}

    pprint(params)
    print()
    print()
    
    
    ### create model callbacks save dir and post-train save dir
    dir1 = f'{CHECKPOINTS_DIR}/{model.name}'
    dir2 = f'{FINAL_EPOCH_DIR}/{model.name}'

    for dir in [dir1, dir2]:
        print(dir)
        try:
            os.mkdir(dir)
            print('success')
        except FileExistsError:
            print('!!! exist')
        except:
            print('~~~ error')
        print()

        
    print()
    ### model callbacks save dir
    if am_I_using_colab:
        epoch5_filepath = '%s/%s/%s.weights.{epoch:03d}.hdf5' % (CHECKPOINTS_DIR, model.name, model.name)
        monitor_filepath = '%s/%s/%s.weights.{epoch:03d}_{val_categorical_accuracy:.4f}_{val_loss:.4f}.hdf5' %
                            (CHECKPOINTS_DIR, model.name, model.name)
        tensorboard_dir = '%s/%s/%s.tensorboard' % (CHECKPOINTS_DIR, model.name, model.name)
    else:
        epoch5_filepath = '%s/%s/%s.weights.{epoch:03d}.hdf5' % (CHECKPOINTS_DIR, model.name, model.name)
        monitor_filepath = '%s/%s/%s.weights.{epoch:03d}_{val_categorical_accuracy:.4f}_{val_loss:.4f}.hdf5' %
                            (CHECKPOINTS_DIR, model.name, model.name)
        tensorboard_dir = '%s/%s/%s.tensorboard' % (CHECKPOINTS_DIR, model.name, model.name)

    # Model's callback: ModelCheckpoint
    epoch_checkpoint = keras.callbacks.ModelCheckpoint(
        filepath=epoch5_filepath,
        monitor='val_loss',
        save_weights_only=True,
        save_freq=int(5 * params['steps_per_epoch'])
        )

    acc_checkpoint = keras.callbacks.ModelCheckpoint(
        filepath=monitor_filepath,
        monitor='val_categorical_accuracy',
        # verbose=1,
        save_best_only=True,
        mode='auto',
        save_weights_only=True,
        save_freq='epoch'
        )

    loss_checkpoint = keras.callbacks.ModelCheckpoint(
        filepath=monitor_filepath,
        monitor='val_loss',
        # verbose=1,
        save_best_only=True,
        mode='auto',
        save_weights_only=True,
        save_freq='epoch'
        )

    # initialize tqdm callback with default parameters
    # tqdm_callback = tfa.callbacks.TQDMProgressBar(
    #     metrics_separator=', '
    #     )

    tensorboard_callback = keras.callbacks.TensorBoard(
        log_dir=tensorboard_dir,
        write_graph=True,
        write_images=True
        )

    callbacks = [
        epoch_checkpoint,
        acc_checkpoint,
        loss_checkpoint,
        # tqdm_callback,
        tensorboard_callback
        ]
    
    
    ### model.fit() verbosity
    if am_I_using_colab:
        verbosity = 2  # end of epoch
    else:
        verbosity = 1  # every step in epoch
        
        
    ### model.fit()
    print(model.name)
    mulai_hitung_waktu()  ###

    training_history = model.fit(
        train_data_gen,
        shuffle=True,
        verbose=verbosity,  # 0 nope, 1 realtime, 2 epoch end
        steps_per_epoch=total_train // batch_size,
        epochs=epochs,
        validation_data=val_data_gen,
        validation_steps=total_val // batch_size,
        callbacks=callbacks,
        workers=0
        )

    cetak_lama_waktu()  ###
    print()
    print()
    
    
    ### post-train save
    history_df = pd.DataFrame(training_history.history)

    history_savepath = f'{FINAL_EPOCH_DIR}/{model.name}/history.{model.name}.csv'
    weights_savepath = f'{FINAL_EPOCH_DIR}/{model.name}/weights.{model.name}.h5'
    model_savepath = f'{FINAL_EPOCH_DIR}/{model.name}/model.{model.name}.h5'


    # save to csv: 
    with open(history_savepath, mode='w') as f:
        history_df.to_csv(
            f,
            header=True,
            index=False
            )

    model.save_weights(
        filepath=weights_savepath,
        overwrite=True,
        save_format='h5'
        )

    model.save(
        filepath=model_savepath,
        overwrite=True,
        include_optimizer=True,
        save_format='h5'
        )
    
    
    ### rechecking save dir
    _checkpoints_listdir = os.listdir(f'{CHECKPOINTS_DIR}/{model.name}')
    print(len(_checkpoints_listdir))
    pprint(_checkpoints_listdir)
    
    _final_epoch_listdir = os.listdir(f'{FINAL_EPOCH_DIR}/{model.name}')
    print(len(_final_epoch_listdir))
    pprint(_final_epoch_listdir)
    
    if am_I_using_colab:
        !cp -fRv "/content/checkpoints" "/content/drive/MyDrive/MyNotebook/MySkripsi"
        print()
        !cp -fRv "/content/weights and models" "/content/drive/MyDrive/MyNotebook/MySkripsi"
        print()
        print('copy success')
    else:
        pass
        
    print()
    print()
    if _checkpoints_listdir and _final_epoch_listdir:
        print('all process done, please recheck before terminating runtime session')
    else:
        print('WARNING : some files failed to save')
    print()
    print('''don't forget to save the model.fit() verbose output''')
    
    
    keyvalpairs = {
        'name' : model.name,
        'history_df' : history_df,
        'model' : model,
        'checkpoints_listdir' : _checkpoints_listdir,
        'final_epoch_listdir' : _final_epoch_listdir
    }
    
    print()
    print()
    print('Returned values using %d bytes of memory now' % (sys.getsizeof(keyvalpairs),))
    return keyvalpairs

In [15]:
# global variable
epochs = 50

In [16]:
# input_size,  # [32, 64, 96, 128] or divisible by 32
# batch_size,  # [16, 32, 64] or power of 
# weights,     # [None, 'imagenet']
# dense        # [True, False]

In [17]:
# mobilenetv2_1.00_96-64-12
model_14 = consecutiveModelTraining(
    input_size=96,
    batch_size=64,
    weights=None,
    dense=False)

Found 9072 images belonging to 12 classes.
Found 6048 images belonging to 12 classes.
mobilenetv2_1.00_96-64-12


Model: "mobilenetv2_1.00_96-64-12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 96, 96, 3)]       0         
_________________________________________________________________
mobilenetv2_1.00_96 (Functio (None, 3, 3, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 12)                15372     
Total params: 2,273,356
Trainable params: 2,239,244
Non-trainable params: 34,112
____________________________________________

In [18]:
model_14['history_df'].describe()

,loss,categorical_accuracy,top-3,ROC-AUC,PR-AUC,precision,recall,TP,TN,FP,...,val_categorical_accuracy,val_top-3,val_ROC-AUC,val_PR-AUC,val_precision,val_recall,val_TP,val_TN,val_FP,val_FN
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,...,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.065536,0.980500,0.995049,0.997692,0.990837,0.986164,0.977460,8804.960000,98978.080000,109.920000,...,0.988128,0.998906,0.999245,0.997090,0.989524,0.987068,5938.200000,66113.520000,62.480000,77.800000
std,0.195863,0.058963,0.028922,0.011279,0.045653,0.028310,0.073435,661.500982,166.849418,166.849418,...,0.016416,0.001974,0.001284,0.006008,0.013939,0.018359,110.447328,81.905431,81.905431,110.447328
min,0.001726,0.581816,0.794849,0.919784,0.676088,0.806289,0.478242,4308.000000,98053.000000,6.000000,...,0.902926,0.989528,0.993000,0.963066,0.919477,0.888298,5344.000000,65708.000000,3.000000,3.000000
25%,0.015568,0.986151,0.999112,0.999103,0.996640,0.987507,0.984791,8871.000000,98975.750000,40.250000,...,0.988863,0.998629,0.999380,0.997774,0.990387,0.987907,5943.250000,66118.250000,14.500000,20.250000
50%,0.026385,0.992174,0.999556,0.999540,0.998422,0.993054,0.991952,8935.500000,99025.500000,62.500000,...,0.992769,0.999668,0.999639,0.998812,0.993592,0.992271,5969.500000,66137.500000,38.500000,46.500000
75%,0.048854,0.995282,0.999778,0.999752,0.999267,0.995529,0.994755,8960.750000,99047.750000,112.250000,...,0.997174,1.000000,0.999908,0.999691,0.997588,0.996634,5995.750000,66161.500000,57.750000,72.750000
max,1.391766,0.999334,1.000000,1.000000,0.999999,0.999334,0.999334,9002.000000,99082.000000,1035.000000,...,0.999501,1.000000,1.000000,0.999999,0.999501,0.999501,6013.000000,66173.000000,468.000000,672.000000


In [19]:
# mobilenetv2_1.00_64-16-fc12
model_15 = consecutiveModelTraining(
    input_size=64,
    batch_size=16,
    weights=None,
    dense=True)

Found 9072 images belonging to 12 classes.
Found 6048 images belonging to 12 classes.
mobilenetv2_1.00_64-16-fc12


Model: "mobilenetv2_1.00_64-16-fc12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
mobilenetv2_1.00_64 (Functio (None, 2, 2, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               327936    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               

In [20]:
model_15['history_df'].describe()

,loss,categorical_accuracy,top-3,ROC-AUC,PR-AUC,precision,recall,TP,TN,FP,...,val_categorical_accuracy,val_top-3,val_ROC-AUC,val_PR-AUC,val_precision,val_recall,val_TP,val_TN,val_FP,val_FN
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,...,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.655971,0.774718,0.923587,0.965719,0.838849,0.870819,0.697650,6329.080000,99140.880000,651.120000,...,0.814471,0.946300,0.976675,0.874721,0.906945,0.732431,4429.740000,66214.300000,313.700000,1618.260000
std,0.488612,0.200908,0.096756,0.048062,0.188383,0.117509,0.268375,2434.697952,281.141524,281.141524,...,0.182952,0.067178,0.031439,0.163107,0.097515,0.270853,1638.118094,179.322677,179.322677,1638.118094
min,0.164832,0.174052,0.465388,0.725611,0.191849,0.452685,0.019511,177.000000,98628.000000,214.000000,...,0.281415,0.708829,0.858215,0.320128,0.463389,0.010913,66.000000,65790.000000,47.000000,121.000000
25%,0.240575,0.684579,0.900243,0.958960,0.784879,0.842950,0.560185,5082.000000,98887.500000,395.500000,...,0.721850,0.925678,0.969470,0.816543,0.881195,0.553075,3345.000000,66086.750000,150.500000,313.000000
50%,0.519272,0.846836,0.963073,0.984260,0.917491,0.895508,0.801201,7268.500000,99065.000000,727.000000,...,0.900711,0.976604,0.990333,0.957694,0.929022,0.873512,5283.000000,66217.500000,310.500000,765.000000
75%,0.934500,0.939925,0.985670,0.993855,0.975178,0.954176,0.925898,8399.750000,99396.500000,904.500000,...,0.957176,0.990865,0.996108,0.983194,0.967923,0.948247,5735.000000,66377.500000,441.250000,2703.000000
max,2.208457,0.963294,0.992063,0.996089,0.986136,0.973690,0.955247,8666.000000,99578.000000,1164.000000,...,0.981481,0.995040,0.998622,0.994265,0.986340,0.979993,5927.000000,66481.000000,738.000000,5982.000000


In [18]:
# mobilenetv2_1.00_128-imagenet128-16-12
model_12 = consecutiveModelTraining(
    input_size=128,
    batch_size=16,
    weights='imagenet',
    dense=False)

Found 9072 images belonging to 12 classes.
Found 6048 images belonging to 12 classes.
mobilenetv2_1.00_128-imagenet128-16-12


Model: "mobilenetv2_1.00_128-imagenet128-16-12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_128 (Functi (None, 4, 4, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 12)                15372     
Total params: 2,273,356
Trainable params: 15,372
Non-trainable params: 2,257,984
__________________

Instructions for updating:
use `tf.profiler.experimental.stop` instead.


  2/567 [..............................] - ETA: 3:09 - loss: 2.6868 - categorical_accuracy: 0.2500 - top-3: 0.4062 - ROC-AUC: 0.6475 - PR-AUC: 0.2021 - precision: 0.4545 - recall: 0.1562 - TP: 5.0000 - TN: 346.0000 - FP: 6.0000 - FN: 27.0000          WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.2160s vs `on_train_batch_end` time: 0.4470s). Check your callbacks.


567/567 [==============================] - 204s 359ms/step - loss: 0.2954 - categorical_accuracy: 0.9106 - top-3: 0.9778 - ROC-AUC: 0.9919 - PR-AUC: 0.9667 - precision: 0.9307 - recall: 0.8983 - TP: 8149.0000 - TN: 99185.0000 - FP: 607.0000 - FN: 923.0000 - val_loss: 0.0845 - val_categorical_accuracy: 0.9744 - val_top-3: 0.9985 - val_ROC-AUC: 0.9984 - val_PR-AUC: 0.9942 - val_precision: 0.9761 - val_recall: 0.9735 - val_TP: 5888.0000 - val_TN: 66384.0000 - val_FP: 144.0000 - val_FN: 160.0000
Epoch 2/50
567/567 [==============================] - 199s 350ms/step - loss: 0.1914 - categorical_accuracy: 0.9509 - top-3: 0.9955 - ROC-AUC: 0.9943 - PR-AUC: 0.9807 - precision: 0.9535 - recall: 0.9490 - TP: 8609.0000 - TN: 99372.0000 - FP: 420.0000 - FN: 463.0000 - val_loss: 0.0477 - val_categorical_accuracy: 0.9864 - val_top-3: 0.9995 - val_ROC-AUC: 0.9986 - val_PR-AUC: 0.9960 - val_precision: 0.9866 - val_recall: 0.9861 - val_TP: 5964.0000 - val_TN: 66447.0000 - val_FP: 81.0000 - val_FN: 84.00

In [19]:
model_12['history_df'].describe()

,loss,categorical_accuracy,top-3,ROC-AUC,PR-AUC,precision,recall,TP,TN,FP,...,val_categorical_accuracy,val_top-3,val_ROC-AUC,val_PR-AUC,val_precision,val_recall,val_TP,val_TN,val_FP,val_FN
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,...,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.124728,0.982319,0.998735,0.995722,0.986462,0.982912,0.981971,8908.440000,99637.500000,154.500000,...,0.994851,0.999798,0.998862,0.996400,0.994912,0.994815,6016.640000,66497.240000,30.760000,31.360000
std,0.034930,0.012842,0.003112,0.001087,0.004046,0.010430,0.014417,130.791181,92.241088,92.241088,...,0.004808,0.000341,0.000794,0.002530,0.004633,0.004901,29.639163,27.985827,27.985827,29.639163
min,0.071764,0.910604,0.977844,0.991876,0.966745,0.930676,0.898258,8149.000000,99185.000000,78.000000,...,0.974372,0.998347,0.996399,0.988536,0.976127,0.973545,5888.000000,66384.000000,3.000000,3.000000
25%,0.104082,0.980985,0.998898,0.994951,0.984602,0.981040,0.980985,8899.500000,99620.000000,105.000000,...,0.993882,0.999835,0.998497,0.995319,0.993882,0.993882,6011.000000,66491.000000,13.000000,13.000000
50%,0.121625,0.986607,0.999339,0.995973,0.987225,0.986606,0.986552,8950.000000,99670.500000,121.500000,...,0.996610,0.999835,0.999051,0.996982,0.996610,0.996610,6027.500000,66507.500000,20.500000,20.500000
75%,0.136586,0.988426,0.999669,0.996454,0.989122,0.988426,0.988426,8967.000000,99687.000000,172.000000,...,0.997851,1.000000,0.999374,0.998066,0.997851,0.997851,6035.000000,66515.000000,37.000000,37.000000
max,0.295429,0.991402,0.999890,0.997477,0.991936,0.991402,0.991402,8994.000000,99714.000000,607.000000,...,0.999504,1.000000,0.999910,0.999704,0.999504,0.999504,6045.000000,66525.000000,144.000000,160.000000


In [20]:
# mobilenetv2_1.00_128-imagenet128-16-fc12
model_13 = consecutiveModelTraining(
    input_size=128,
    batch_size=16,
    weights='imagenet',
    dense=True)

Found 9072 images belonging to 12 classes.
Found 6048 images belonging to 12 classes.
mobilenetv2_1.00_128-imagenet128-16-fc12


Model: "mobilenetv2_1.00_128-imagenet128-16-fc12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_128 (Functi (None, 4, 4, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               327936    
_________________________________________________________________
dropout_2 (Dropout)          

567/567 [==============================] - 210s 371ms/step - loss: 0.9332 - categorical_accuracy: 0.7167 - top-3: 0.8922 - ROC-AUC: 0.9583 - PR-AUC: 0.8120 - precision: 0.8418 - recall: 0.6383 - TP: 5791.0000 - TN: 98704.0000 - FP: 1088.0000 - FN: 3281.0000 - val_loss: 0.2159 - val_categorical_accuracy: 0.9339 - val_top-3: 0.9889 - val_ROC-AUC: 0.9967 - val_PR-AUC: 0.9816 - val_precision: 0.9566 - val_recall: 0.9064 - val_TP: 5482.0000 - val_TN: 66279.0000 - val_FP: 249.0000 - val_FN: 566.0000
Epoch 2/50
567/567 [==============================] - 196s 345ms/step - loss: 0.4747 - categorical_accuracy: 0.8571 - top-3: 0.9538 - ROC-AUC: 0.9859 - PR-AUC: 0.9313 - precision: 0.9180 - recall: 0.8137 - TP: 7382.0000 - TN: 99133.0000 - FP: 659.0000 - FN: 1690.0000 - val_loss: 0.1222 - val_categorical_accuracy: 0.9661 - val_top-3: 0.9944 - val_ROC-AUC: 0.9985 - val_PR-AUC: 0.9926 - val_precision: 0.9730 - val_recall: 0.9611 - val_TP: 5813.0000 - val_TN: 66367.0000 - val_FP: 161.0000 - val_FN: 2

In [21]:
model_13['history_df'].describe()

,loss,categorical_accuracy,top-3,ROC-AUC,PR-AUC,precision,recall,TP,TN,FP,...,val_categorical_accuracy,val_top-3,val_ROC-AUC,val_PR-AUC,val_precision,val_recall,val_TP,val_TN,val_FP,val_FN
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,...,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.00000,50.000000,50.000000,50.00000
mean,0.366156,0.898122,0.956537,0.991593,0.959142,0.958711,0.868598,7879.920000,99456.660000,335.340000,...,0.978535,0.993588,0.998941,0.995232,0.990476,0.969177,5861.58000,66472.040000,55.960000,186.42000
std,0.104098,0.031320,0.011824,0.005509,0.024505,0.023122,0.040095,363.737898,167.627577,167.627577,...,0.014725,0.005045,0.001388,0.005533,0.009150,0.020777,125.66088,52.837433,52.837433,125.66088
min,0.239665,0.716711,0.892196,0.958298,0.811982,0.841837,0.638338,5791.000000,98704.000000,130.000000,...,0.924438,0.963955,0.990778,0.966255,0.956552,0.890212,5384.00000,66279.000000,10.000000,38.00000
25%,0.311537,0.893546,0.953180,0.990745,0.956638,0.952694,0.864804,7845.500000,99405.500000,229.500000,...,0.973752,0.992601,0.998960,0.994808,0.989407,0.961682,5816.25000,66465.250000,20.000000,103.50000
50%,0.336975,0.907407,0.958995,0.993344,0.966350,0.965901,0.879905,7982.500000,99511.000000,281.000000,...,0.983052,0.994709,0.999311,0.997096,0.992805,0.975694,5901.00000,66485.000000,43.000000,147.00000
75%,0.397503,0.914324,0.962963,0.994017,0.970635,0.972280,0.887897,8055.000000,99562.500000,386.500000,...,0.987806,0.996197,0.999577,0.998144,0.996623,0.982887,5944.50000,66508.000000,62.750000,231.75000
max,0.933249,0.924272,0.970569,0.995600,0.977540,0.984328,0.900022,8165.000000,99662.000000,1088.000000,...,0.995536,0.998512,0.999883,0.999445,0.998326,0.993717,6010.00000,66518.000000,249.000000,664.00000


In [18]:
# mobilenetv2_1.00_32-16-12
model_5 = consecutiveModelTraining(
    input_size=32,
    batch_size=16,
    weights=None,
    dense=False)

Found 9072 images belonging to 12 classes.
Found 6048 images belonging to 12 classes.
mobilenetv2_1.00_32-16-12


Model: "mobilenetv2_1.00_32-16-12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
mobilenetv2_1.00_32 (Functio (None, 1, 1, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 12)                15372     
Total params: 2,273,356
Trainable params: 2,239,244
Non-trainable params: 34,112
____________________________________________

In [19]:
model_5['history_df'].describe()

,loss,categorical_accuracy,top-3,ROC-AUC,PR-AUC,precision,recall,TP,TN,FP,...,val_categorical_accuracy,val_top-3,val_ROC-AUC,val_PR-AUC,val_precision,val_recall,val_TP,val_TN,val_FP,val_FN
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,...,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.268970,0.915703,0.969436,0.982766,0.938048,0.936689,0.896936,8137.000000,99473.220000,318.780000,...,0.928393,0.977063,0.987454,0.948693,0.943546,0.910982,5509.620000,66333.980000,194.020000,538.380000
std,0.526670,0.174939,0.101987,0.058148,0.170451,0.131785,0.216617,1965.151861,363.387143,363.387143,...,0.156797,0.081915,0.042740,0.151721,0.123135,0.201692,1219.831937,248.775162,248.775162,1219.831937
min,0.028907,0.146384,0.393408,0.637517,0.135074,0.235038,0.023810,216.000000,98236.000000,57.000000,...,0.216931,0.507110,0.737408,0.206371,0.300469,0.021164,128.000000,65494.000000,24.000000,31.000000
25%,0.047280,0.946621,0.990658,0.995089,0.978937,0.957651,0.937224,8502.500000,99416.000000,97.250000,...,0.954944,0.992808,0.996331,0.984651,0.966844,0.943990,5709.250000,66331.500000,47.500000,61.250000
50%,0.077058,0.979718,0.997299,0.998350,0.994425,0.982486,0.977017,8863.500000,99634.000000,158.000000,...,0.981730,0.998347,0.998643,0.995921,0.983173,0.980159,5928.000000,66426.500000,101.500000,120.000000
75%,0.206202,0.987765,0.998677,0.998898,0.996638,0.989249,0.986580,8950.250000,99694.750000,376.000000,...,0.990534,0.998801,0.999340,0.997927,0.992127,0.989873,5986.750000,66480.500000,196.500000,338.750000
max,2.944268,0.993276,0.999339,0.999570,0.998846,0.993712,0.992945,9008.000000,99735.000000,1556.000000,...,0.995370,1.000000,0.999892,0.999517,0.996026,0.994874,6017.000000,66504.000000,1034.000000,5920.000000


In [17]:
# mobilenetv2_1.00_32-imagenet224-32-12
model32imagenet2243212 = consecutiveModelTraining(
    input_size=32,
    batch_size=32,
    weights='imagenet',
    dense=False)

Found 9072 images belonging to 12 classes.
Found 6048 images belonging to 12 classes.
mobilenetv2_1.00_32-imagenet224-32-12


Model: "mobilenetv2_1.00_32-imagenet224-32-12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 1, 1, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 12)                15372     
Total params: 2,273,356
Trainable params: 15,372
Non-trainable params: 2,257,984
____________________

In [18]:
model32imagenet2243212['history_df'].describe()

,loss,categorical_accuracy,top-3,ROC-AUC,PR-AUC,precision,recall,TP,TN,FP,...,val_categorical_accuracy,val_top-3,val_ROC-AUC,val_PR-AUC,val_precision,val_recall,val_TP,val_TN,val_FP,val_FN
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,...,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,1.995568,0.312719,0.548695,0.767203,0.349184,0.734523,0.145973,1319.600000,98959.400000,480.600000,...,0.277808,0.490040,0.724957,0.316195,0.841115,0.113598,687.040000,66397.740000,130.260000,5360.960000
std,0.027282,0.009248,0.009670,0.007806,0.012426,0.017186,0.014708,132.958947,66.806452,66.806452,...,0.007638,0.007400,0.005081,0.008356,0.013265,0.008221,49.722889,17.139059,17.139059,49.722889
min,1.967080,0.265819,0.496239,0.720744,0.279530,0.709899,0.063606,575.000000,98888.000000,119.000000,...,0.250165,0.472884,0.710097,0.288584,0.816934,0.080192,485.000000,66368.000000,73.000000,5297.000000
25%,1.983956,0.311062,0.544939,0.765510,0.346147,0.724686,0.144275,1304.250000,98929.500000,471.000000,...,0.274595,0.486111,0.722878,0.312607,0.831584,0.112310,679.250000,66386.750000,125.000000,5330.500000
50%,1.991128,0.314878,0.550608,0.768618,0.351374,0.734396,0.149336,1350.000000,98946.000000,494.000000,...,0.278356,0.491485,0.725888,0.317326,0.839162,0.114583,693.000000,66394.500000,133.500000,5355.000000
75%,2.000022,0.316344,0.554923,0.770877,0.355241,0.739891,0.152848,1381.750000,98969.000000,510.500000,...,0.282325,0.495040,0.727998,0.322270,0.848285,0.118634,717.500000,66403.000000,141.250000,5368.750000
max,2.157672,0.324226,0.559292,0.775261,0.362253,0.828530,0.157743,1426.000000,99321.000000,552.000000,...,0.296627,0.512401,0.733933,0.332623,0.871585,0.124173,751.000000,66455.000000,160.000000,5563.000000


In [19]:
# mobilenetv2_1.00_32-imagenet224-64-12
model32imagenet2246412 = consecutiveModelTraining(
    input_size=32,
    batch_size=64,
    weights='imagenet',
    dense=False)

Found 9072 images belonging to 12 classes.
Found 6048 images belonging to 12 classes.
mobilenetv2_1.00_32-imagenet224-64-12


Model: "mobilenetv2_1.00_32-imagenet224-64-12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 1, 1, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1280)              0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 12)                15372     
Total params: 2,273,356
Trainable params: 15,372
Non-trainable params: 2,257,984
____________________

In [20]:
model32imagenet2246412['history_df'].describe()

,loss,categorical_accuracy,top-3,ROC-AUC,PR-AUC,precision,recall,TP,TN,FP,...,val_categorical_accuracy,val_top-3,val_ROC-AUC,val_PR-AUC,val_precision,val_recall,val_TP,val_TN,val_FP,val_FN
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,...,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,1.987698,0.315007,0.552877,0.769386,0.352096,0.748137,0.142320,1282.020000,98651.060000,436.940000,...,0.278225,0.492410,0.725594,0.317195,0.855271,0.110618,665.480000,66062.360000,113.640000,5350.520000
std,0.034435,0.010603,0.011944,0.009475,0.014181,0.020363,0.017118,154.197526,75.057912,75.057912,...,0.008190,0.007699,0.006168,0.009055,0.017571,0.009764,58.739911,21.091405,21.091405,58.739911
min,1.958833,0.253885,0.480240,0.709217,0.266001,0.718869,0.046070,415.000000,98571.000000,76.000000,...,0.250166,0.468418,0.700570,0.282074,0.821983,0.069149,416.000000,66027.000000,40.000000,5286.000000
25%,1.974665,0.313999,0.551621,0.769347,0.350603,0.737521,0.141791,1277.250000,98611.500000,435.500000,...,0.275474,0.488198,0.723724,0.315070,0.845174,0.108544,653.000000,66049.000000,104.500000,5318.000000
50%,1.980890,0.316940,0.554341,0.770769,0.354837,0.745077,0.146536,1320.000000,98633.500000,454.500000,...,0.279837,0.494016,0.726763,0.317919,0.854129,0.113863,685.000000,66062.000000,114.000000,5331.000000
75%,1.990420,0.320493,0.557837,0.772796,0.358222,0.752498,0.150089,1352.000000,98652.500000,476.500000,...,0.284034,0.496676,0.729411,0.323205,0.865935,0.116024,698.000000,66071.500000,127.000000,5363.000000
max,2.198852,0.324156,0.565386,0.776340,0.363975,0.845214,0.155639,1402.000000,99012.000000,517.000000,...,0.290392,0.507812,0.734753,0.327935,0.912281,0.121343,730.000000,66136.000000,149.000000,5600.000000


In [21]:
# mobilenetv2_1.00_96-imagenet96-64-fc12
model96imagenet64_64fc12 = consecutiveModelTraining(
    input_size=96,
    batch_size=64,
    weights='imagenet',
    dense=True)

Found 9072 images belonging to 12 classes.
Found 6048 images belonging to 12 classes.
mobilenetv2_1.00_96-imagenet96-64-fc12


Model: "mobilenetv2_1.00_96-imagenet96-64-fc12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 96, 96, 3)]       0         
_________________________________________________________________
mobilenetv2_1.00_96 (Functio (None, 3, 3, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_3 ( (None, 1280)              0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               327936    
_________________________________________________________________
dropout_7 (Dropout)          (Non

In [22]:
model96imagenet64_64fc12['history_df'].describe()

,loss,categorical_accuracy,top-3,ROC-AUC,PR-AUC,precision,recall,TP,TN,FP,...,val_categorical_accuracy,val_top-3,val_ROC-AUC,val_PR-AUC,val_precision,val_recall,val_TP,val_TN,val_FP,val_FN
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,...,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.169084,0.958435,0.988366,0.995815,0.984503,0.973262,0.947345,8533.680000,98857.280000,230.720000,...,0.990066,0.998295,0.999350,0.997777,0.993209,0.986872,5937.020000,66135.520000,40.480000,78.980000
std,0.103625,0.032032,0.014384,0.004433,0.019862,0.015455,0.040820,367.709926,112.259183,112.259183,...,0.007812,0.001182,0.000483,0.002073,0.006260,0.009766,58.750767,36.950734,36.950734,58.750767
min,0.098848,0.751776,0.890764,0.966454,0.852486,0.887926,0.682504,6148.000000,98312.000000,122.000000,...,0.951130,0.994182,0.998017,0.988098,0.959709,0.942320,5669.000000,65938.000000,13.000000,29.000000
25%,0.125838,0.960119,0.989260,0.995843,0.985297,0.972251,0.948684,8545.750000,98844.500000,170.250000,...,0.989279,0.998005,0.999238,0.997568,0.993031,0.985622,5929.500000,66134.250000,21.000000,47.000000
50%,0.143546,0.964476,0.990786,0.996743,0.989111,0.977612,0.956428,8615.500000,98890.500000,197.500000,...,0.992769,0.998504,0.999457,0.998422,0.995161,0.990691,5960.000000,66147.000000,29.000000,56.000000
75%,0.177937,0.969638,0.991868,0.997225,0.990604,0.980658,0.961923,8665.000000,98917.750000,243.500000,...,0.994141,0.999003,0.999671,0.999034,0.996494,0.992188,5969.000000,66155.000000,41.750000,86.500000
max,0.815178,0.977464,0.994671,0.998237,0.993317,0.986238,0.970582,8743.000000,98966.000000,776.000000,...,0.996343,0.999668,0.999902,0.999695,0.997826,0.995180,5987.000000,66163.000000,238.000000,347.000000


In [23]:
# mobilenetv2_1.00_128-16-fc12
model_8 = consecutiveModelTraining(
    input_size=128,
    batch_size=16,
    weights=None,
    dense=True)

Found 9072 images belonging to 12 classes.
Found 6048 images belonging to 12 classes.
mobilenetv2_1.00_128-16-fc12


Model: "mobilenetv2_1.00_128-16-fc12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 128, 128, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_128 (Functi (None, 4, 4, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_4 ( (None, 1280)              0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 1280)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)               327936    
_________________________________________________________________
dropout_11 (Dropout)         (None, 256)             

In [24]:
model_8['history_df'].describe()

,loss,categorical_accuracy,top-3,ROC-AUC,PR-AUC,precision,recall,TP,TN,FP,...,val_categorical_accuracy,val_top-3,val_ROC-AUC,val_PR-AUC,val_precision,val_recall,val_TP,val_TN,val_FP,val_FN
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,...,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.557772,0.819173,0.945736,0.973354,0.876842,0.888860,0.767077,6958.920000,99057.920000,734.080000,...,0.864967,0.962887,0.982361,0.912823,0.922567,0.822024,4971.600000,66193.180000,334.820000,1076.400000
std,0.421236,0.167134,0.087133,0.041186,0.156959,0.072527,0.226870,2058.165538,305.860235,305.860235,...,0.163544,0.074923,0.033868,0.144888,0.078954,0.230768,1395.685948,187.131045,187.131045,1395.685948
min,0.192106,0.252205,0.542108,0.770561,0.272393,0.627400,0.079255,719.000000,98425.000000,266.000000,...,0.281415,0.601852,0.812064,0.314717,0.535615,0.095734,579.000000,65672.000000,94.000000,137.000000
25%,0.289302,0.796021,0.958609,0.978488,0.877081,0.853737,0.737020,6686.250000,98892.250000,504.500000,...,0.846023,0.976521,0.988695,0.926056,0.907529,0.825066,4990.000000,66078.750000,176.000000,241.250000
50%,0.402250,0.890597,0.978560,0.988437,0.940599,0.908135,0.874173,7930.500000,99092.000000,700.000000,...,0.929398,0.988509,0.993720,0.969584,0.940096,0.920139,5565.000000,66231.500000,296.500000,483.000000
75%,0.627780,0.925375,0.985808,0.992153,0.966691,0.938878,0.912864,8281.500000,99287.500000,899.750000,...,0.962880,0.995618,0.997065,0.987122,0.968208,0.960111,5806.750000,66352.000000,449.250000,1058.000000
max,2.063020,0.946759,0.990741,0.995255,0.980379,0.969562,0.933973,8473.000000,99526.000000,1367.000000,...,0.980820,0.997355,0.998499,0.994991,0.983527,0.977348,5911.000000,66434.000000,856.000000,5469.000000


In [25]:
# mobilenetv2_1.00_64-16-12
model_9 = consecutiveModelTraining(
    input_size=64,
    batch_size=16,
    weights=None,
    dense=False)

Found 9072 images belonging to 12 classes.
Found 6048 images belonging to 12 classes.
mobilenetv2_1.00_64-16-12


Model: "mobilenetv2_1.00_64-16-12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 64, 64, 3)]       0         
_________________________________________________________________
mobilenetv2_1.00_64 (Functio (None, 2, 2, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_5 ( (None, 1280)              0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 1280)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 12)                15372     
Total params: 2,273,356
Trainable params: 2,239,244
Non-trainable params: 34,112
____________________________________________

In [26]:
model_9['history_df'].describe()

,loss,categorical_accuracy,top-3,ROC-AUC,PR-AUC,precision,recall,TP,TN,FP,...,val_categorical_accuracy,val_top-3,val_ROC-AUC,val_PR-AUC,val_precision,val_recall,val_TP,val_TN,val_FP,val_FN
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,...,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.123873,0.961215,0.987557,0.993094,0.975879,0.971624,0.954189,8656.400000,99612.800000,179.200000,...,0.970675,0.994444,0.996965,0.984995,0.977431,0.963819,5829.180000,66417.120000,110.880000,218.820000
std,0.335667,0.111861,0.065078,0.034364,0.106444,0.068567,0.136865,1241.638187,243.231644,243.231644,...,0.074463,0.023064,0.011452,0.064273,0.047118,0.105453,637.776869,143.274055,143.274055,637.776869
min,0.010871,0.252425,0.543320,0.758012,0.271373,0.550476,0.095569,867.000000,98343.000000,30.000000,...,0.481812,0.836640,0.918505,0.545133,0.679066,0.259590,1570.000000,65786.000000,18.000000,20.000000
25%,0.026504,0.977045,0.997933,0.998358,0.994144,0.979097,0.974234,8838.250000,99603.250000,56.500000,...,0.979456,0.997396,0.998470,0.994761,0.981953,0.976480,5905.750000,66419.250000,33.000000,42.250000
50%,0.042775,0.988040,0.998898,0.999116,0.997027,0.989394,0.987158,8955.500000,99696.000000,96.000000,...,0.988757,0.998843,0.999005,0.997127,0.989735,0.987517,5972.500000,66466.000000,62.000000,75.500000
75%,0.081123,0.992725,0.999421,0.999451,0.998299,0.993755,0.992201,9001.250000,99735.500000,188.750000,...,0.993965,0.999504,0.999551,0.998695,0.994535,0.993014,6005.750000,66495.000000,108.750000,142.250000
max,2.261839,0.996473,0.999780,0.999934,0.999746,0.996692,0.996252,9038.000000,99762.000000,1449.000000,...,0.996858,1.000000,0.999905,0.999687,0.997021,0.996693,6028.000000,66510.000000,742.000000,4478.000000


In [27]:
# mobilenetv2_1.00_96-imagenet96-16-fc12
model_10 = consecutiveModelTraining(
    input_size=96,
    batch_size=16,
    weights='imagenet',
    dense=True)

Found 9072 images belonging to 12 classes.
Found 6048 images belonging to 12 classes.
mobilenetv2_1.00_96-imagenet96-16-fc12


Model: "mobilenetv2_1.00_96-imagenet96-16-fc12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 96, 96, 3)]       0         
_________________________________________________________________
mobilenetv2_1.00_96 (Functio (None, 3, 3, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_6 ( (None, 1280)              0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 1280)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 256)               327936    
_________________________________________________________________
dropout_16 (Dropout)         (Non

In [28]:
model_10['history_df'].describe()

,loss,categorical_accuracy,top-3,ROC-AUC,PR-AUC,precision,recall,TP,TN,FP,...,val_categorical_accuracy,val_top-3,val_ROC-AUC,val_PR-AUC,val_precision,val_recall,val_TP,val_TN,val_FP,val_FN
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.00000,50.00000,...,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.385921,0.893064,0.955825,0.991078,0.957694,0.958878,0.860157,7803.340000,99460.54000,331.46000,...,0.974778,0.992229,0.998685,0.994419,0.990643,0.960860,5811.280000,66473.280000,54.720000,236.720000
std,0.097510,0.028443,0.012780,0.005589,0.023404,0.016727,0.036251,328.867296,113.24368,113.24368,...,0.017607,0.005750,0.001426,0.006917,0.006688,0.025456,153.959714,39.131406,39.131406,153.959714
min,0.297434,0.707011,0.871914,0.955243,0.804753,0.866166,0.622795,5650.000000,98919.00000,218.00000,...,0.877976,0.965939,0.992766,0.953789,0.976666,0.823909,4983.000000,66390.000000,10.000000,82.000000
25%,0.333054,0.892113,0.954723,0.990974,0.957690,0.958183,0.856481,7770.000000,99453.75000,271.50000,...,0.969039,0.991030,0.998607,0.993638,0.986416,0.951802,5756.500000,66447.000000,22.250000,144.250000
50%,0.369886,0.896715,0.958113,0.992068,0.961564,0.964038,0.866237,7858.500000,99496.50000,295.50000,...,0.979001,0.993469,0.999134,0.996069,0.993038,0.965691,5840.500000,66487.000000,41.000000,207.500000
75%,0.415976,0.902943,0.959656,0.993309,0.966236,0.966901,0.873291,7922.500000,99520.50000,338.25000,...,0.985739,0.995660,0.999592,0.998132,0.996229,0.976149,5903.750000,66505.750000,81.000000,291.500000
max,0.963861,0.918651,0.966490,0.995071,0.973012,0.973033,0.890653,8080.000000,99574.00000,873.00000,...,0.991567,0.998677,0.999928,0.999249,0.998297,0.986442,5966.000000,66518.000000,138.000000,1065.000000
